# Fourier transformations of audio signals

The Fourier transformation is a method of transforming between the frequency and time domains.

$$ f(t) \leftrightarrow F(\omega) $$

such that

$$ F(\omega) = \mathcal{F}(f(t))= \int_{-\infty}^{\infty} \mathcal{f}(t) \exp(-i\omega t) dt$$

This is a fundamental method for signal decomposition and its descrete implementation is important for practical signal processing and can be found in many other applications.

The purpose of this project is to gain experience with the use of the Fourier transform 

1. Apply the Fourier transform on a number of .wav sound files for examination of the component frequencies
1. Filter out some component frequencies to examine results

Further goals may be to

1. Implement the Fourier transform to work with a input stream from a audio recording device using the [`Soundevice`](https://python-sounddevice.readthedocs.io/en/0.5.1/) python library 